In [1]:
import py_stringsimjoin as ssj
from py_stringsimjoin.debugmatcher.debug_matcher import *
from py_stringsimjoin.labeler.labeler import *
from py_stringsimjoin.active_learner.active_learner import *
from sklearn.linear_model import LogisticRegression
import pandas as pd
import py_stringmatching as sm
from py_stringsimjoin.sampler.sample import *
from py_stringsimjoin.feature.autofeaturegen import *
from py_stringsimjoin.feature.extractfeatures import *
from py_stringsimjoin.matcher.matcher_utils import split_train_test

In [2]:
ws=sm.WhitespaceTokenizer(return_set=True)
qg3=sm.QgramTokenizer(qval=3,return_set=True)

In [3]:
ldf=pd.read_csv('/scratch/songs/msd_reduced.csv')
rdf=ldf.copy()

In [67]:
ldf[ldf['title']=='Majareta (directo 05)']
ildf=ldf.set_index('id')
ildf.ix[119124]

title                 Majareta (directo 05)
release                          En directo
artist_name                         La Fuga
duration                            176.222
artist_familiarity                 0.604755
artist_hotttnesss                  0.444876
year                                      0
Name: 119124, dtype: object

In [4]:
import re
#ldf['cln_title'] = ldf['title'].apply(lambda r: re.sub(r'[^\w\s]',' ',str(r).lower()))
#rdf['cln_title'] = rdf['title'].apply(lambda r: re.sub(r'[^\w\s]',' ',str(r).lower()))
comp_rx = re.compile('[^\w\s]')
l=[]
for idx, row in ldf.iterrows():
    s=re.sub(comp_rx,' ',str(row['title']).lower())
    if pd.notnull(row['artist_name']):
        s += re.sub(comp_rx,' ',str(row['artist_name']).lower())
    l.append(s)
r=[]
for idx, row in rdf.iterrows():
    s=re.sub(comp_rx,' ',str(row['title']).lower())
    if pd.notnull(row['artist_name']):
        s += re.sub(comp_rx,' ',str(row['artist_name']).lower())
    r.append(s)
ldf['cln_title'] = l
rdf['cln_title'] = r

In [5]:
seed=pd.read_csv('/scratch/songs/msdSeed.csv')

In [6]:
c=sample_pairs(ldf,rdf,'id','id','cln_title','cln_title',100000,50,seed)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:03:37


In [7]:
len(c)

100004

In [8]:
labeled_c = label_table_using_gold(c, 'l_id', 'r_id', '/scratch/msdGold_v1.csv')
sum(labeled_c['label'])

2564

In [9]:
ft=get_features()
ft.drop('dice_ws', inplace=True)
ft.drop('cosine_ws', inplace=True)
ft.drop('overlap_ws', inplace=True)
ft.drop('overlap_coefficient_ws', inplace=True)

In [10]:
#ft.ix['jaccard_ws']['tokenizer'] = qg3
fvs = extract_feature_vecs(c, 'l_id','r_id',ldf,rdf,'id','id','cln_title','cln_title',ft,n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:20


In [11]:
len(fvs)

100004

In [12]:
lr=LogisticRegression()
al=ActiveLearner(lr,20,20,'/scratch/msdGold_v1.csv',seed)

In [13]:
lp = al.learn(fvs, '_id', 'l_id','r_id')

          l_id    r_id  jaccard_ws
_id                               
100000  276262  155485         1.0
100001      40      41         0.0
100002  615227  784865         1.0
100003      28      50         0.0
          l_id    r_id  jaccard_ws  label
_id                                      
100000  276262  155485         1.0      1
100001      40      41         0.0      0
100002  615227  784865         1.0      1
100003      28      50         0.0      0
Selecting next batch...
computing entropy
sorting
sampling
[0.6931465380961338, 0.6931465380961338, 0.69314603222502558, 0.69314603222502558, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.69314584321105688, 0.6931458432110568

py_stringsimjoin/active_learner/active_learner.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  to_be_labeled_pairs['label'] = labels


In [14]:
lp.head()

,l_id,r_id,jaccard_ws,label
_id,,,,
100000,276262,155485,1.00,1
100001,40,41,0.00,0
100002,615227,784865,1.00,1
100003,28,50,0.00,0
1691,529250,668485,0.25,0


In [15]:
lp.reset_index(level=0, inplace=True)
sum(lp['label'])

197

In [16]:
reload(ssj.matcher.matcher_utils)
splits = split_train_test(lp, 'label')
P=splits['train']
Q=splits['test']

In [17]:
len(P), len(Q)
sum(P['label']), sum(Q['label'])

(98, 99)

In [18]:
P['l_id'] = P['l_id'].astype(int)
P['r_id'] = P['r_id'].astype(int)
Q['l_id'] = Q['l_id'].astype(int)
Q['r_id'] = Q['r_id'].astype(int)

In [19]:
P[P['jaccard_ws'] < 0.5].head()
ldf.dtypes
P.dtypes
P

,_id,l_id,r_id,jaccard_ws,label
30,1174,268452,268452,1.000000,1
187,47470,269625,44269,0.750000,0
178,3470,144564,144564,1.000000,1
286,2142,61937,61937,1.000000,1
26,3227,765647,949457,1.000000,1
201,2326,300856,560223,1.000000,1
338,22663,889246,333209,0.666667,0
282,5810,390170,390170,1.000000,1
102,1806,471435,471435,1.000000,1
62,89314,972452,635957,0.538462,0


In [20]:
reload(ssj.debugmatcher.debug_matcher)
lr1=LogisticRegression()
dbg = debug_matcher(lr1, P, Q, 'jaccard_ws', 'label', 'l_id', 'r_id', ldf, rdf, 'id', 'id')

0.817789382838


In [39]:
dbg['false_pos']

,_id,l_id,r_id,jaccard_ws,label,l_title,l_release,l_artist_name,l_duration,l_artist_familiarity,...,l_year,l_cln_title,r_title,r_release,r_artist_name,r_duration,r_artist_familiarity,r_artist_hotttnesss,r_year,r_cln_title
160,71004,206733,665116,0.846154,0,My Little Girl's Gone (1997 Digital Remaster) (Mono),In Touch With Peter And Gordon,Peter And Gordon,153.31220,0.567192,...,0,my little girl s gone 1997 digital remaster mono peter and gordon,My Little Girl's Gone (1997 Digital Remaster) (Stereo),In Touch With Peter And Gordon,Peter And Gordon,153.25995,0.567192,0.382570,0,my little girl s gone 1997 digital remaster stereo peter and gordon
122,89786,686695,262124,0.833333,0,Life Goes On [Albin Meyers Remix],Subliminal Essentials 2009 mixed by Richard Grey,Richard Grey vs Erick Morillo and Jose Nunez feat Shawnee Taylor,445.33506,0.586649,...,0,life goes on albin meyers remix richard grey vs erick morillo and jose nunez feat shawnee taylor,Life Goes On (2008 Remix),Strictly Erick Morillo,Richard Grey vs Erick Morillo and Jose Nunez feat Shawnee Taylor,468.66240,0.650128,0.330777,0,life goes on 2008 remix richard grey vs erick morillo and jose nunez feat shawnee taylor
170,15581,584654,519880,0.857143,0,Fluid Drive (Alt Take 2),Complete Debut Recordings,John LaPorta,202.00444,0.266716,...,0,fluid drive alt take 2 john laporta,Fluid Drive (Alt Take),Complete Debut Recordings,John LaPorta,213.78567,0.266716,0.000000,0,fluid drive alt take john laporta
289,6185,994907,539236,1.000000,0,Long Live Rock 'N' Roll,Finyl Vinyl,Rainbow,427.41506,0.700636,...,1978,long live rock n rollrainbow,Long Live Rock N Roll,Classic,Rainbow,260.70159,0.700636,0.511546,2005,long live rock n rollrainbow
134,78167,416935,570177,0.857143,0,I Still Dream (Part I),Conflict,Falling Cycle,140.85179,0.471614,...,0,i still dream part i falling cycle,I Still Dream (Part II),Conflict,Falling Cycle,168.64608,0.471614,0.311371,0,i still dream part ii falling cycle
336,9399,707838,739959,0.909091,0,In All The Wrong Places Instrumental (12 inch version),In All the Wrong Places/ Give Thanks,Kero One,469.15873,0.591749,...,0,in all the wrong places instrumental 12 inch version kero one,In All The Wrong Places (12 inch version),In All the Wrong Places/ Give Thanks,Kero One,469.15873,0.455079,0.431345,0,in all the wrong places 12 inch version kero one
121,40368,735791,350881,0.833333,0,Mahler: Quartet for Piano and Strings in A minor: Scherzo,String Quartet No.3/Piano Quartet/Piano Quintet,Borodin String Quartet,382.11873,0.395970,...,0,mahler quartet for piano and strings in a minor scherzoborodin string quartet,Mahler: Quartet for Piano and Strings in A minor: Allegro,String Quartet No.3/Piano Quartet/Piano Quintet,Borodin String Quartet,694.28200,0.395970,0.334351,0,mahler quartet for piano and strings in a minor allegroborodin string quartet
180,64857,895118,639628,0.818182,0,Handel - Sonata No. 6 In E Major For Violin & Harpsichord_ Op.1 No.15 - II - Allegro,Simply Handel 1,Josef Suk - Violin and Zuzana Ruzicková - Harpsichord,174.44526,0.368250,...,0,handel sonata no 6 in e major for violin harpsichord_ op 1 no 15 ii allegrojosef suk ...,Handel - Sonata No. 6 In E Major For Violin & Harpsichord_ Op.1 No.15 - III - Largo,Simply Handel 1,Josef Suk - Violin and Zuzana Ruzicková - Harpsichord,117.47220,0.368250,0.331725,0,handel sonata no 6 in e major for violin harpsichord_ op 1 no 15 iii largojosef suk v...
125,877,60218,861885,1.000000,0,Hey Baby,Dirty Dancing,Bruce Channel,143.43791,0.547171,...,1987,hey babybruce channel,Hey! Baby,The Greatest Rock 'n' Roll Hits,Bruce Channel,144.24771,0.547171,0.445912,1962,hey babybruce channel


In [22]:
dbg['stats']

{'f1': 0.9463414634146342,
 'precision': 0.9150943396226415,
 'recall': 0.9797979797979798}

In [23]:
-lr1.intercept_/lr1.coef_

array([[ 0.81778938]])

In [24]:
len(dbg['false_neg']), len(dbg['false_pos'])

(2, 9)

In [54]:
lr2=LogisticRegression()
lr2.fit(lp[['jaccard_ws']].values, lp['label'].values)
-lr2.intercept_/lr2.coef_

array([[ 0.83868252]])

In [33]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
lsvc.fit(lp[['jaccard_ws']].values, lp['label'].values)
-lsvc.intercept_/lsvc.coef_

array([[ 0.86037635]])

In [28]:
what_if_matrix(lp, 'jaccard_ws', 'label', 0.86, 0.02, 10)

,threshold,precision,recall,f1
0,0.66,0.544199,1.000000,0.704830
1,0.68,0.601227,0.994924,0.749522
2,0.70,0.616352,0.994924,0.761165
3,0.72,0.717472,0.979695,0.828326
4,0.74,0.752941,0.974619,0.849558
5,0.76,0.845815,0.974619,0.905660
6,0.78,0.888889,0.974619,0.929782
7,0.80,0.893023,0.974619,0.932039
8,0.82,0.945813,0.974619,0.960000
9,0.84,0.960000,0.974619,0.967254


In [34]:
cc=ssj.jaccard_join(ldf,rdf,'id','id','cln_title','cln_title',ws,0.83868252)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:56


In [35]:
len(cc)

1281172

In [36]:
labeled_cc = label_table_using_gold(cc, 'l_id', 'r_id', '/scratch/msdGold_v1.csv')
sum(labeled_cc['label'])


1271628

In [38]:
1271628/1281172.0, 1271628/1292023.0

(0.9925505708835347, 0.9842146772928965)